Full code to make all waypoints from road sections for video demo

In [4]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.15/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

# connect to the sim 
client = carla.Client('localhost', 2000)

world = client.get_world()

spectator = world.get_spectator()

# get map look at the map
town_map = world.get_map()
roads = town_map.get_topology()

# set up route generations
sampling_resolution = 1
grp = GlobalRoutePlanner(town_map, sampling_resolution)

# show first 10 road sections animated in a loop in an example
for i in range(10):
    spectator_pos = carla.Transform(roads[i][0].transform.location + carla.Location(x=0,y=0,z=80),
                                carla.Rotation(yaw = roads[i][0].transform.rotation.yaw +0,pitch = roads[i][0].transform.rotation.pitch -90))
    spectator.set_transform(spectator_pos)
    time.sleep(0.1)
    cur_route = grp.trace_route(roads[i][0].transform.location,roads[i][1].transform.location)
    for wp in cur_route:
        if len(cur_route)<20:
            durn = 2
        else:
            durn = 0.8
        world.debug.draw_string(wp[0].transform.location, '^', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0), life_time=0.8,
            persistent_lines=True)
        time.sleep(0.03)
    if len(cur_route)<20:
        time.sleep(1)

In [14]:
# make unique waypoints and show on the map 
# accumulate all waypoints all roads in a loop while making them unique on location x an y
unique_waypoints = []
for road in roads:
    
    cur_route = grp.trace_route(road[0].transform.location,road[1].transform.location)
    for wp in cur_route:
        if len(unique_waypoints)==0:
            unique_waypoints.append(wp[0]) #first waypoint is added regardless to start the list
        else:
            found = False
            for uwp in unique_waypoints: #check for same located waypoints and ignore if found
                if abs(uwp.transform.location.x - wp[0].transform.location.x) < 0.1 \
                            and abs(uwp.transform.location.y - wp[0].transform.location.y)<0.1 \
                            and abs(uwp.transform.rotation.yaw - wp[0].transform.rotation.yaw)<20:
                    found = True
            if not found:
                unique_waypoints.append(wp[0])

#let's show them on the map
for uwp in unique_waypoints:
    world.debug.draw_string(uwp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

#move spectator for top down view to see all points 
spectator_pos = carla.Transform(carla.Location(x=0,y=30,z=200),
                                carla.Rotation(pitch = -90, yaw = -90))
spectator.set_transform(spectator_pos)

show gaps

In [15]:
'''
another way - explore what we got in town_map = world.get_map()
this is a better option:
1. it allows to set a distance between waypoints
2. it gets them all without gaps
3. Do not need ti use route planner

#this takes about 4 minutes - in RL context, this only needs to run at the start or when you change a towns
'''
all_waypoints = town_map.generate_waypoints(0.3) #note 0.3 is distance between waypoints in meters

# make unique
unique_waypoints = []
for wp in all_waypoints:
    if len(unique_waypoints)==0:
        unique_waypoints.append(wp) #first waypoint is added regardless to start the list
    else:
        found = False
        for uwp in unique_waypoints: #check for same located waypoints and ignore if found
            if abs(uwp.transform.location.x - wp.transform.location.x) < 0.1 \
                            and abs(uwp.transform.location.y - wp.transform.location.y)<0.1 \
                            and abs(uwp.transform.rotation.yaw - wp.transform.rotation.yaw)<20:  #this checks same direction
                found = True
                break
        if not found:
            unique_waypoints.append(wp)

# draw all point in the sim for 60 seconds
for wp in unique_waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)

#move spectator for top down view to see all points 
spectator_pos = carla.Transform(carla.Location(x=0,y=30,z=200),
                                carla.Rotation(pitch = -90, yaw = -90))
spectator.set_transform(spectator_pos)

In [31]:
'''
now we will see how this will be used in real examples, like during RL training
e.g. see how long it takes to find closest point to the car

'''
#remove any cars/clean up the sim
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

#drop teh car from spectator position
vehicle_bp = world.get_blueprint_library().filter('*model3*')
vehicle = world.try_spawn_actor(vehicle_bp[0], spectator.get_transform())
time.sleep(10)


In [35]:
# this takes less than 0.0 seconds, i.e. it happens in something like 0.001 or similar

my_waypoint = vehicle.get_transform().location
curr_distance = 1000
for wp in unique_waypoints:
    dist = my_waypoint.distance(wp.transform.location)
    if dist < curr_distance:
        curr_distance =  dist
        selected_wp = wp

#draw the waypoint
world.debug.draw_string(selected_wp.transform.location, '^', draw_shadow=False,
    color=carla.Color(r=0, g=0, b=255), life_time=60.0,
    persistent_lines=True)

In [36]:
# example how to use this for measuring where the car is in relation to where it should be

vehicle_transform = vehicle.get_transform()
distance_to_wp = selected_wp.transform.location.distance(vehicle_transform.location)
direction_difference = (vehicle_transform.rotation.yaw - selected_wp.transform.rotation.yaw) % 180
print('deviation from waypoint:',distance_to_wp,'metres')
print('angle discrepancy:',direction_difference,'degrees')


deviation from waypoint: 5.731203556060791 metres
angle discrepancy: 42.31391143798828 degrees
